# Simply Parse HiResPerf Log and Read the Raw Counters.

Namely, this script doesn't process the data to calculate the rate of counters.

Right now, this is mainly used for `instructed_profile`, where we only log twice during each function running, one at the beginning and one at the ending.

In [4]:
import numpy as np
import pandas as pd

In [97]:
cpu_store_imc = 0

In [9]:
def read_logs_to_numpy(file_path: str, use_imc: bool = False, use_offcore: bool = False) -> np.ndarray:
    c1_name = 'offcore_read' if use_offcore else 'llc_misses'
    c2_name = 'offcore_write' if use_offcore else 'sw_prefetch'

    if use_imc:
        # Matches "iQQQQQQQQ" -> int32, uint64, uint64, ...
        dt = np.dtype([
            ('cpu_id', np.int32),
            ('timestamp', np.uint64),
            ('stall_mem', np.uint64),
            ('inst_retire', np.uint64),
            ('cpu_unhalt', np.uint64),
            (f'{c1_name}', np.uint64),
            (f'{c2_name}', np.uint64),
            ('imc_read', np.uint64),
            ('imc_write', np.uint64),
        ])
    else:
        # Matches "iQQQQQQ"
        dt = np.dtype([
            ('cpu_id', np.int32),
            ('timestamp', np.uint64),
            ('stall_mem', np.uint64),
            ('inst_retire', np.uint64),
            ('cpu_unhalt', np.uint64),
            (f'{c1_name}', np.uint64),
            (f'{c2_name}', np.uint64),
        ])

    print("Reading binary file into NumPy array...")
    try:
        data = np.fromfile(file_path, dtype=dt)
        print(f"Read {len(data)} records to successfully.")
        return data
    except Exception as e:
        print(f"Error reading file with NumPy: {e}")
        return np.array([])


In [10]:
data_array = read_logs_to_numpy("/hrperf_log.bin", use_imc=True, use_offcore=True)

Reading binary file into NumPy array...
Read 94 records to successfully.


In [11]:
df = pd.DataFrame.from_records(data_array)
df.head()

,cpu_id,timestamp,stall_mem,inst_retire,cpu_unhalt,offcore_read,offcore_write,imc_read,imc_write
0,0,656571153406265,1062090235256,140743433727982,2902498307713,8489389044,22061383522,44585607,24211094
1,1,656571153406265,440773286282,142470805231537,1425522258565,2839518030,5036135684,0,0
2,2,656571153406265,393272438829,140802933730719,1198164590757,4544465103,4658329135,0,0
3,3,656571153406265,354116036558,140800235833997,1153837191974,3202279105,4187522633,0,0
4,4,656571153406265,780475143787,140792347498704,1883794308336,12267296589,6479459308,0,0


In [101]:
def calc_data_in_range(time_range: tuple[int, int], df: pd.DataFrame, use_offcore: bool) -> pd.DataFrame:
    c1_name = 'offcore_read' if use_offcore else 'llc_misses'
    c2_name = 'offcore_write' if use_offcore else 'sw_prefetch'
    c1_diff_name = f'{c1_name}_diff'
    c2_diff_name = f'{c2_name}_diff'

    start, end = time_range
    mask = (df['timestamp'] >= start) & (df['timestamp'] <= end)
    df_in_range = df.loc[mask]
    assert len(df_in_range["timestamp"].unique()) == 2, "There should be exactly two unique timestamps in the specified range."
    timestamp_min = df_in_range['timestamp'].min()
    timestamp_max = df_in_range['timestamp'].max()

    data_at_start = df.loc[df['timestamp'] == timestamp_min, ['cpu_id', f'{c1_name}', f'{c2_name}']]
    data_at_end = df.loc[df['timestamp'] == timestamp_max, ['cpu_id', f'{c1_name}', f'{c2_name}']]


    merged = pd.merge(data_at_end, data_at_start, on='cpu_id', suffixes=('_end', '_start'))
    
    print(f"{len(merged)} CPUs valid record found in the specified time range.")

    diff = pd.DataFrame({
        'cpu_id': merged['cpu_id'],
        f'{c1_diff_name}': merged['offcore_read_end'] - merged['offcore_read_start'],
        f'{c2_diff_name}': merged['offcore_write_end'] - merged['offcore_write_start']
    })
    print(f"Duration: {(timestamp_max - timestamp_min)/1999/1e6:.5f} s")
    
    diff_sum = diff[[c1_diff_name, c2_diff_name]].sum()
    print(diff_sum.to_dict())
    
    total_ = diff_sum.sum()
    print(f"Total diff: {total_}")
    print(f"Data transferred ((c1+c2) * 64): {total_ * 64 / 1e6:.2f} MB") 
    
    imc_col_names = ['imc_read', 'imc_write']

    start_imc = df.loc[(df['timestamp'] == timestamp_min) & (df['cpu_id'] == cpu_store_imc), imc_col_names]
    end_imc = df.loc[(df['timestamp'] == timestamp_max) & (df['cpu_id'] == cpu_store_imc), imc_col_names]

    if not start_imc.empty and not end_imc.empty:
        imc_read_diff = int(end_imc['imc_read'].values[0]) - int(start_imc['imc_read'].values[0])
        imc_write_diff = int(end_imc['imc_write'].values[0]) - int(start_imc['imc_write'].values[0])
        print(f"IMC read diff: {imc_read_diff}, IMC write diff: {imc_write_diff}")
        print(f"IMC total transferred ((read+write) * 64): {(imc_read_diff + imc_write_diff) * 64 / 1e6:.2f} MB")
    else:
        print("IMC data not available for the specified cpu_id and timestamps.")

    return df_in_range

In [102]:
df["timestamp"].min(), df["timestamp"].max()

(np.uint64(656571153406265), np.uint64(656674223845409))

In [103]:
calc_data_in_range((df["timestamp"].min(), df["timestamp"].max()), df, use_offcore=True)

46 CPUs valid record found in the specified time range.
Duration: 51.56100 s
{'offcore_read_diff': 57279306, 'offcore_write_diff': 77881107}
Total diff: 135160413
Data transferred ((c1+c2) * 64): 8650.27 MB
IMC read diff: 104907910, IMC write diff: 56410481
IMC total transferred ((read+write) * 64): 10324.38 MB


,cpu_id,timestamp,stall_mem,inst_retire,cpu_unhalt,offcore_read,offcore_write,imc_read,imc_write
0,0,656571153406265,1062090235256,140743433727982,2902498307713,8489389044,22061383522,44585607,24211094
1,1,656571153406265,440773286282,142470805231537,1425522258565,2839518030,5036135684,0,0
2,2,656571153406265,393272438829,140802933730719,1198164590757,4544465103,4658329135,0,0
3,3,656571153406265,354116036558,140800235833997,1153837191974,3202279105,4187522633,0,0
4,4,656571153406265,780475143787,140792347498704,1883794308336,12267296589,6479459308,0,0
...,...,...,...,...,...,...,...,...,...
89,43,656674223845409,468787233048,141977549310694,1497602305447,5038054080,4358925014,0,0
90,44,656674223845409,338998473889,142317680760262,1067701930669,2563844062,1407048069,0,0
91,45,656674223845409,531028783865,141815657198931,1615027336625,5936804698,6492449649,0,0
92,46,656674223845409,438871261174,140787163803558,1957735271616,5497596359,3481349218,0,0
